<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Exploring-OpenAI" data-toc-modified-id="Exploring-OpenAI-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Exploring OpenAI</a></span></li></ul></div>

# Reinforcement Learning <a class="tocSkip">

## Exploring OpenAI

In [1]:
#!pip install gym

In [2]:
#!pip install git+https://github.com/jakevdp/JSAnimation.git

In [3]:
import numpy as np
np.random.seed(123)
print("NumPy:{}".format(np.__version__))

import tensorflow as tf
tf.set_random_seed(123)
print("TensorFlow:{}".format(tf.__version__))

import keras
print("Keras:{}".format(keras.__version__))

import gym
print('OpenAI Gym:',gym.__version__)

NumPy:1.14.5
TensorFlow:1.11.0
Keras:2.2.4
OpenAI Gym: 0.10.8


Using TensorFlow backend.


# OpenAI 101

In [4]:
# print how many environments available

all_env = list(gym.envs.registry.all())

print('Total Environments in Gym version {} : {}'.format(gym.__version__,len(all_env)))

Total Environments in Gym version 0.10.8 : 797


In [5]:
for e in list(all_env):
    print(e)

EnvSpec(Copy-v0)
EnvSpec(RepeatCopy-v0)
EnvSpec(ReversedAddition-v0)
EnvSpec(ReversedAddition3-v0)
EnvSpec(DuplicatedInput-v0)
EnvSpec(Reverse-v0)
EnvSpec(CartPole-v0)
EnvSpec(CartPole-v1)
EnvSpec(MountainCar-v0)
EnvSpec(MountainCarContinuous-v0)
EnvSpec(Pendulum-v0)
EnvSpec(Acrobot-v1)
EnvSpec(LunarLander-v2)
EnvSpec(LunarLanderContinuous-v2)
EnvSpec(BipedalWalker-v2)
EnvSpec(BipedalWalkerHardcore-v2)
EnvSpec(CarRacing-v0)
EnvSpec(Blackjack-v0)
EnvSpec(KellyCoinflip-v0)
EnvSpec(KellyCoinflipGeneralized-v0)
EnvSpec(FrozenLake-v0)
EnvSpec(FrozenLake8x8-v0)
EnvSpec(CliffWalking-v0)
EnvSpec(NChain-v0)
EnvSpec(Roulette-v0)
EnvSpec(Taxi-v2)
EnvSpec(GuessingGame-v0)
EnvSpec(HotterColder-v0)
EnvSpec(Reacher-v2)
EnvSpec(Pusher-v2)
EnvSpec(Thrower-v2)
EnvSpec(Striker-v2)
EnvSpec(InvertedPendulum-v2)
EnvSpec(InvertedDoublePendulum-v2)
EnvSpec(HalfCheetah-v2)
EnvSpec(Hopper-v2)
EnvSpec(Swimmer-v2)
EnvSpec(Walker2d-v2)
EnvSpec(Ant-v2)
EnvSpec(Humanoid-v2)
EnvSpec(HumanoidStandup-v2)
EnvSpec(FetchS

# Play the cartploe game with stochastic control

In [11]:
import matplotlib.pyplot as plt
import matplotlib.animation as anm
from matplotlib import rc
rc('animation', html='html5')
#rc('animation', ffmpeg_path='/usr/bin/ffmpeg')
%matplotlib inline


In [12]:

from matplotlib import animation
from JSAnimation.IPython_display import display_animation


In [13]:
def env_render(env_vis):
    plt.figure()
    plot = plt.imshow(env_vis[0])
    plt.axis('off')
    def animate(i):
        plot.set_data(env_vis[i])

    anim = anm.FuncAnimation(plt.gcf(),
                             animate,
                             frames=len(env_vis),
                             interval=20,
                             repeat=True,
                             repeat_delay=20)
    display(display_animation(anim, default_mode='loop'))


In [17]:

    
env = gym.make('CartPole-v0')
n_episodes = 1
env_vis = []
for i_episode in range(n_episodes):
    observation = env.reset()
    for t in range(100):
        #env_vis.append(env.render(mode = 'rgb_array'))
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished at t{}".format(t+1))
            break
#env.render(close=True)
#env_render(env_vis)

[ 0.04173573  0.0193333  -0.00014932 -0.00517414]
[ 4.21223935e-02  2.14457393e-01 -2.52805889e-04 -2.97904180e-01]
[ 0.04641154  0.40958295 -0.00621089 -0.59066683]
[ 0.0546032   0.6047913  -0.01802423 -0.8852997 ]
[ 0.06669903  0.80015327 -0.03573022 -1.18359389]
[ 0.08270209  0.60551266 -0.0594021  -0.90232176]
[ 0.09481234  0.80138685 -0.07744853 -1.21306878]
[ 0.11084008  0.6073451  -0.10170991 -0.94562577]
[ 0.12298698  0.803679   -0.12062242 -1.26845583]
[ 0.13906056  0.61028623 -0.14599154 -1.01585153]
[ 0.15126629  0.80702119 -0.16630857 -1.35058392]
[ 0.16740671  1.00379534 -0.19332025 -1.69034233]
Episode finished at t12


# Simple Policies

In [18]:
def policy_logic(env,obs):
    return 1 if obs[2] > 0 else 0
def policy_random(env,obs):
    return env.action_space.sample()

def experiment(policy, n_episodes, rewards_max):
    rewards=np.empty(shape=(n_episodes))
    env = gym.make('CartPole-v0')
    
    for i in range(n_episodes):
        obs = env.reset()
        done = False
        episode_reward = 0
        while not done:
            action = policy(env,obs)
            obs, reward, done, info = env.step(action)
            episode_reward += reward
            if episode_reward > rewards_max:
                break
        rewards[i]=episode_reward
        #print("Episode finished at t{}".format(reward))
    print('Policy:{}, Min reward:{}, Max reward:{}, Average reward:{}'
          .format(policy.__name__,
                  np.min(rewards),
                  np.max(rewards),
                  np.mean(rewards)))

n_episodes = 100
rewards_max = 10000
experiment(policy_random, n_episodes, rewards_max)
experiment(policy_logic, n_episodes, rewards_max)

Policy:policy_random, Min reward:9.0, Max reward:72.0, Average reward:23.35
Policy:policy_logic, Min reward:25.0, Max reward:61.0, Average reward:41.92


# Simple Policies with Parameters

In [19]:
def policy_logic(theta,obs):
    # just ignore theta
    return 1 if obs[2] > 0 else 0

def policy_random(theta,obs):
    return 0 if np.matmul(theta,obs) < 0 else 1

def episode(env, policy, rewards_max):
    obs = env.reset()
    done = False
    episode_reward = 0
    if policy.__name__ in ['policy_random']:
        theta = np.random.rand(4) * 2 - 1
    else:
        theta = None
    while not done:
        action = policy(theta,obs)
        obs, reward, done, info = env.step(action)
        episode_reward += reward
        if episode_reward > rewards_max:
            break
    return episode_reward
    
def experiment(policy, n_episodes, rewards_max):
    rewards=np.empty(shape=(n_episodes))
    env = gym.make('CartPole-v0')
    
    for i in range(n_episodes):
        rewards[i]=episode(env,policy,rewards_max)
        #print("Episode finished at t{}".format(reward))
    print('Policy:{}, Min reward:{}, Max reward:{}, Average reward:{}'
          .format(policy.__name__,
                  np.min(rewards),
                  np.max(rewards),
                  np.mean(rewards)))

n_episodes = 100
rewards_max = 10000
experiment(policy_random, n_episodes, rewards_max)
experiment(policy_logic, n_episodes, rewards_max)

Policy:policy_random, Min reward:8.0, Max reward:200.0, Average reward:38.44
Policy:policy_logic, Min reward:25.0, Max reward:68.0, Average reward:41.8


# Simple Policies with Training

In [20]:
# train with random search

def policy_logic(theta,obs):
    # just ignore theta
    return 1 if obs[2] > 0 else 0

def policy_random(theta,obs):
    return 0 if np.matmul(theta,obs) < 0 else 1

def episode(env,policy, rewards_max,theta):
    obs = env.reset()
    done = False
    episode_reward = 0

    while not done:
        action = policy(theta,obs)
        obs, reward, done, info = env.step(action)
        episode_reward += reward
        if episode_reward > rewards_max:
            break
    return episode_reward

def train(policy, n_episodes, rewards_max):

    env = gym.make('CartPole-v0')
    np.random.seed(0)
    
    theta_best = np.empty(shape=[4])
    reward_best = 0

    for i in range(n_episodes):
        if policy.__name__ in ['policy_random']:
            theta = np.random.rand(4) * 2 - 1
        else:
            theta = None
        
        reward_episode=episode(env,policy,rewards_max, theta)
        if reward_episode > reward_best:
            reward_best = reward_episode
            theta_best = theta.copy()
    return reward_best,theta_best
    
def experiment(policy, n_episodes, rewards_max, theta=None):
    rewards=np.empty(shape=[n_episodes])
    env = gym.make('CartPole-v0')
    
    for i in range(n_episodes):
        rewards[i]=episode(env,policy,rewards_max,theta)
        #print("Episode finished at t{}".format(reward))
    print('Policy:{}, Min reward:{}, Max reward:{}, Average reward:{}'
          .format(policy.__name__,
                  np.min(rewards),
                  np.max(rewards),
                  np.mean(rewards)))

n_episodes = 100
rewards_max = 10000

reward,theta = train(policy_random, n_episodes, rewards_max)
print('trained theta: {}, rewards: {}'.format(theta,reward))
experiment(policy_random, n_episodes, rewards_max, theta)

experiment(policy_logic, n_episodes, rewards_max)

trained theta: [-0.1526904   0.29178823 -0.12482558  0.783546  ], rewards: 200.0
Policy:policy_random, Min reward:200.0, Max reward:200.0, Average reward:200.0
Policy:policy_logic, Min reward:24.0, Max reward:64.0, Average reward:41.7


# Simple Policies with Training until Certain Rewards

In [21]:
# train with random search until we reach rewards > 200
def policy_logic(theta,obs):
    # just ignore theta
    return 1 if obs[2] > 0 else 0

def policy_random(theta,obs):
    return 0 if np.matmul(theta,obs) < 0 else 1

def episode(env,policy, rewards_max,theta):
    obs = env.reset()
    done = False
    episode_reward = 0

    while not done:
        action = policy(theta,obs)
        obs, reward, done, info = env.step(action)
        episode_reward += reward
        if episode_reward > rewards_max:
            break
    return episode_reward

def train(policy, n_episodes, rewards_max):

    env = gym.make('CartPole-v0')
    np.random.seed(0)
    
    theta_best = np.empty(shape=[4])
    reward_best = 0

    # n_episodes acts as a max in this case
    for i in range(n_episodes):
        if policy.__name__ in ['policy_random']:
            theta = np.random.rand(4) * 2 - 1
        else:
            theta = None
        reward_episode=episode(env,policy,rewards_max, theta)
        if reward_episode > reward_best:
            reward_best = reward_episode
            theta_best = theta.copy()
        if reward_best >= rewards_max:
            break
    return reward_best,theta_best
    
def experiment(policy, n_episodes, rewards_max, theta=None):
    rewards=np.empty(shape=[n_episodes])
    env = gym.make('CartPole-v0')
    
    for i in range(n_episodes):
        rewards[i]=episode(env,policy,rewards_max,theta)
        #print("Episode finished at t{}".format(reward))
    print('Policy:{}, Min reward:{}, Max reward:{}, Average reward:{}'
          .format(policy.__name__,
                  np.min(rewards),
                  np.max(rewards),
                  np.mean(rewards)))

n_episodes = 10000 # acts as max episodes to use in training
rewards_max = 200
reward,theta = train(policy_random, n_episodes, rewards_max)
print('trained theta: {}, rewards: {}'.format(theta,reward))

n_episodes = 100
rewards_max = 10000
experiment(policy_random, n_episodes, rewards_max, theta)
experiment(policy_logic, n_episodes, rewards_max)

trained theta: [-0.1526904   0.29178823 -0.12482558  0.783546  ], rewards: 200.0
Policy:policy_random, Min reward:200.0, Max reward:200.0, Average reward:200.0
Policy:policy_logic, Min reward:25.0, Max reward:68.0, Average reward:42.19


# Neural Network Policy

In [22]:
# train with the neural network

def policy_random(theta,obs):
    return 0 if np.matmul(theta,obs) < 0 else 1

def policy_naive_nn(nn,obs):
    return np.argmax(nn.predict(np.array([obs])))


# returns obs -> actions -> rewards arrays
# specify t_max to run for t_max steps
# specify r_max to run until r_max is reached
# specify both t_max ad r_max to run for t_max but break if r_max is reached
def episode(env, policy, theta, r_max=0, t_max=0, return_hist_reward=0):
    if return_hist_reward > 0:
        o_list=[]
        a_list=[]
        r_list=[]
    
    episode_reward = 0

    obs = env.reset()
    done = False
    t = 0
    while not done:
        action = policy(theta,obs)
        if return_hist_reward>0:
            o_list.append(obs)
            a_list.append(action)
        obs, reward, done, info = env.step(action)
        if return_hist_reward>0:
            r_list.append(reward)
        episode_reward += reward
        if r_max > 0 and episode_reward > r_max:
            break
        t+=1
        if t_max > 0 and t == t_max:
            break

    
    if return_hist_reward>=episode_reward:
        return_val = [np.array(o_list),np.array(a_list),np.array(r_list)]
    else:
        return_val = episode_reward
    return return_val

# experiment collect observations and rewards for each episode
def experiment(env, policy, n_episodes,theta=None, r_max=0, t_max=0, return_hist_reward=0):
    
    if return_hist_reward>0:
        obs_list=[]
        action_list=[]
    else:
        rewards=np.empty(shape=[n_episodes])
    for i in range(n_episodes):
        val = episode(env,policy,theta, r_max, t_max,return_hist_reward)
        if isinstance(val, list):
            obs_list.append(val[0])
            action_list.append(val[1])
            if not return_hist_reward:
                rewards[i]=np.sum(val[2])
        else:
            if not return_hist_reward:
                rewards[i]=val
            
    if return_hist_reward>0:
        return_val = [np.concatenate(obs_list,axis=0), np.concatenate(action_list,axis=0)]
    else:
        return_val = []
        print('Policy:{}, Min reward:{}, Max reward:{}, Average reward:{}'
          .format(policy.__name__,
                  np.min(rewards),
                  np.max(rewards),
                  np.mean(rewards)))

    return return_val

In [23]:
# build the model
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(8,input_dim=4, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 40        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 18        
Total params: 58
Trainable params: 58
Non-trainable params: 0
_________________________________________________________________


In [24]:
# create training data
env = gym.make('CartPole-v0')
n_obs = 4
n_actions = 2
theta = np.random.rand(4) * 2 - 1
n_episodes = 100
r_max = 0
t_max = 0

x_train, y_train = experiment(env, 
                              policy_random, 
                              n_episodes,
                              theta, 
                              r_max, 
                              t_max, 
                              return_hist_reward=100 )
y_train = np.eye(n_actions)[y_train]
print(x_train.shape,y_train.shape)

(3306, 4) (3306, 2)


In [25]:
# train the model
model.fit(x_train, y_train, epochs=50, batch_size=10)

Epoch 1/50
3306/3306 [==============================] - 2s 531us/step - loss: 0.5677
Epoch 2/50
3306/3306 [==============================] - 1s 273us/step - loss: 0.3952
Epoch 3/50
3306/3306 [==============================] - 1s 274us/step - loss: 0.2616
Epoch 4/50
3306/3306 [==============================] - 1s 275us/step - loss: 0.1903
Epoch 5/50
3306/3306 [==============================] - 1s 275us/step - loss: 0.1510
Epoch 6/50
3306/3306 [==============================] - 1s 272us/step - loss: 0.1271
Epoch 7/50
3306/3306 [==============================] - 1s 273us/step - loss: 0.1109
Epoch 8/50
3306/3306 [==============================] - 1s 273us/step - loss: 0.0990
Epoch 9/50
3306/3306 [==============================] - 1s 273us/step - loss: 0.0899
Epoch 10/50
3306/3306 [==============================] - 1s 275us/step - loss: 0.0827
Epoch 11/50
3306/3306 [==============================] - 1s 274us/step - loss: 0.0767
Epoch 12/50
3306/3306 [==============================] - 1s 273

In [27]:
n_episodes = 200
r_max = 0
t_max = 0

_ = experiment(env, 
              policy_naive_nn, 
              n_episodes,
              theta=model, 
              r_max=r_max, 
              t_max=t_max, 
              return_hist_reward=0 )

_ = experiment(env, 
              policy_random, 
              n_episodes,
              theta, 
              r_max, 
              t_max, 
              return_hist_reward=0 )

Policy:policy_naive_nn, Min reward:73.0, Max reward:189.0, Average reward:114.62
Policy:policy_random, Min reward:77.0, Max reward:190.0, Average reward:116.29
